In [19]:
import os
from dotenv import  load_dotenv
load_dotenv()
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [20]:
from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD,refresh_schema=False)
graph

In [21]:
## Dataset movie
movie_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director,'|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors,'|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres,'|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [22]:
movie_query

"\nLOAD CSV WITH HEADERS FROM \n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director,'|') |\n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors,'|') |\n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres,'|') |\n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n"

In [23]:
graph.query(movie_query)

[]

In [24]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {title: STRING, released: DATE, id: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [25]:
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="gemma2:2b"
)


In [27]:
from langchain_neo4j import GraphCypherQAChain

In [28]:
from langchain_core.prompts import PromptTemplate

# Custom prompt that explicitly tells the LLM not to use markdown
CYPHER_GENERATION_TEMPLATE = """Task: Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
IMPORTANT: Do not wrap the Cypher statement in markdown code blocks (```). Return only the raw Cypher query.

Examples (optional):
{examples}

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question", "examples"],
    template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT
)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x000001E32828E6C0>, cypher_generation_chain=PromptTemplate(input_variables=['examples', 'question', 'schema'], input_types={}, partial_variables={}, template='Task: Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\nIMPORTANT: Do not wrap the Cypher statement in markdown code blocks (```). Return only the raw Cypher query.\n\nExamples (optional):\n{examples}\n\nThe question is:\n{question}')
| RunnableBinding(bound=ChatOllama(model='gemma2:2b'), kwargs={}, config={}, c

In [29]:
response = chain.invoke({'query':"Who was the director of movie casino"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (movie:Movie{title: "Casino"})-[directed]->(person) RETURN person.name 

Full Context:
[{'person.name': 'Drama'}, {'person.name': 'Crime'}]

> Finished chain.


In [30]:
response['result']


'Martin Scorsese directed the movie Casino. \n'